# Season 16 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [90]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

In [91]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 16, Day 7**

---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decree will pass this season.

**Anytime a Player accumulates 400 Runs (scoring as a baserunner or allowing as a pitcher), they'll earn an Item Crate for a random active Player on their Team.**

**Anytime a Division accumulates 99 Wins, each Team in that Division will receive an Item Crate for a random active Player on their Team.**

**Anytime 3000 Runs are accumulated league-wide, every Team will receive an Item Crate for a random active Player on their Team.**

All this season's decrees specify the rates at which players and teams earn Item Crates, which will provide additional
items.

TODO: Analysis

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Plunder
**Steal a Player from the Idols board. Send a random Player from your Team back.**

Due to the random nature of the target on our team, this tests the best case (swapping the Best Player on the Idol Board
for a position with our worst player in that position) and the worst case (swapping the Best Player on the Idol Board
for a position with our best player in that position).

**Best Case Swap:**

In [92]:
idols = [x.player for x in Idol.load().values() if x.player.league_team_id != pies]

idols.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_idol = idols[0]
idols.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_idol = idols[0]

worst_batter = sort_lineup(pies, 1)[0]
display(Markdown(f"Batting: Swap {best_batting_idol.name} for {worst_batter.name}"))

pandas.DataFrame(trade_players(pies, worst_batter, best_batting_idol))

Batting: Swap Valentine Games for Yusef Puddles

,Pies
batting_change,+12.7525%
pitching_change,+0.0000%
baserunning_change,+7.8614%
defense_change,-2.0998%


In [93]:
worst_pitcher = sort_rotation(pies, 1)[0]
display(Markdown(f"Pitching: Swap {best_pitching_idol.name} for {worst_pitcher.name}"))

pandas.DataFrame(trade_players(pies, worst_pitcher, best_pitching_idol))


Pitching: Swap Pitching Machine for Zack Sanders

,Pies
batting_change,+0.0000%
pitching_change,+14.4822%
baserunning_change,+0.0000%
defense_change,+0.0000%


**Worst Case Swap:**

In [94]:
best_batter = sort_lineup(pies, 1, order="best")[0]
display(Markdown(f"Batting: Swap {best_batting_idol.name} for {best_batter.name}"))

pandas.DataFrame(trade_players(pies, best_batter, best_batting_idol))

Batting: Swap Valentine Games for Jaxon Buck-ey

,Pies
batting_change,+0.0525%
pitching_change,+0.0000%
baserunning_change,+3.2516%
defense_change,-1.2209%


In [95]:
best_pitcher = sort_rotation(pies, 1, order="best")[0]
display(Markdown(f"Pitching: Swap {best_pitching_idol.name} for {best_pitcher.name}"))

pandas.DataFrame(trade_players(pies, best_pitcher, best_pitching_idol))


Pitching: Swap Pitching Machine for Nerd Pacheco

,Pies
batting_change,+0.0000%
pitching_change,+8.3449%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Equivalent Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team. They must be within 2 combined stars of each other.**

See below for the 10 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [96]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)

In [97]:
all_total_stars = get_total_stars(all_wild)
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:10]))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:10]), maxVal=1))

####Eduardo Woodman

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Jessica Telephone,5.5,2.5,4.0,3.0
Jacob Haynes,5.5,3.0,2.0,1.5
Math Velazquez,5.0,3.5,4.0,2.5
Comfort Septemberish,5.0,1.0,5.0,2.5
Baby Triumphant,4.5,2.0,2.5,3.0
Scores Baserunner,4.5,1.5,3.0,4.0
Yusef Fenestrate,4.5,2.5,4.0,3.5
August Sky,4.5,4.0,2.0,2.5
Dudley Mueller,4.5,2.5,3.0,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.833060,1.019391,0.846540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Jessica Telephone,1.129590,1.113282,1.216455,1.326294,0.982475,0.215902,1.574396,0.100000,0.919039
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Math Velazquez,0.981884,1.027667,1.067639,0.709586,1.146343,0.395484,0.622834,0.060000,0.682863
Comfort Septemberish,0.978574,1.177996,1.187902,0.375357,0.767285,0.611622,1.105479,0.060000,0.832023
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Scores Baserunner,0.919824,1.152613,0.934063,0.610101,0.478674,0.001000,0.426670,0.001000,0.369985
Yusef Fenestrate,0.917904,0.707016,1.167877,0.770614,1.241785,0.311063,1.179928,0.001000,0.803840
August Sky,0.896214,1.023915,0.872428,0.931996,0.703057,0.321205,0.386864,0.001000,0.389760
Dudley Mueller,0.889823,1.434186,0.605426,0.775583,0.813133,0.409998,0.809489,0.191011,1.459612


####Beasley Day

,Batting,Pitching,Baserunning,Defense
Beasley Day,4.5,3.0,3.5,3.0
Jessica Telephone,5.5,2.5,4.0,3.0
Baby Doyle,5.5,2.5,3.0,4.5
Ve-salius Sundae,5.5,3.0,4.0,3.0
Jacob Haynes,5.5,3.0,2.0,1.5
Margarito Nava,5.0,3.5,4.0,2.5
Gerund Pantheocide,5.0,3.5,3.0,4.0
Math Velazquez,5.0,3.5,4.0,2.5
Comfort Septemberish,5.0,1.0,5.0,2.5
Hiroto Cerna,5.0,4.0,4.0,3.0


...

,Batting,Pitching,Baserunning,Defense
Yusef Puddles,2.0,2.5,3.0,3.5
Jacob Haynes,5.5,3.0,2.0,1.5
Baby Triumphant,4.5,2.0,2.5,3.0
Bauer Zimmerman,4.5,1.5,3.0,2.5
Denzel Scott,4.5,2.5,2.5,2.5
Peanutiel Duffy,4.0,3.5,1.5,3.5
Rat Mason,4.0,1.5,2.5,3.0
Paula Turnip,4.0,1.0,3.5,3.0
Paula Mason,4.0,2.5,3.0,2.0
Wanda Schenn,4.0,2.5,2.5,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Denzel Scott,0.850411,0.963515,0.849616,0.566107,0.627399,0.119743,0.673793,0.007747,0.949707
Peanutiel Duffy,0.841420,1.038584,0.738454,0.996131,0.670202,0.418180,0.343720,0.100000,0.240592
Rat Mason,0.831824,1.089116,0.635958,0.783859,0.732174,0.207444,0.899090,0.019604,0.485050
Paula Turnip,0.816815,0.766132,1.017299,0.583585,0.885149,0.544858,0.284335,0.100000,0.261251
Paula Mason,0.776714,0.652438,0.982483,0.483644,0.608965,0.010000,0.827057,0.100000,0.277885
Wanda Schenn,0.773087,0.507807,1.188279,1.204166,0.468774,0.037260,0.318748,0.724941,0.409040


####Usurper Violet

,Batting,Pitching,Baserunning,Defense
Usurper Violet,3.0,3.0,1.5,3.5
Jacob Haynes,5.5,3.0,2.0,1.5
Baby Triumphant,4.5,2.0,2.5,3.0
August Sky,4.5,4.0,2.0,2.5
Bauer Zimmerman,4.5,1.5,3.0,2.5
Denzel Scott,4.5,2.5,2.5,2.5
Peanutiel Duffy,4.0,3.5,1.5,3.5
Rat Mason,4.0,1.5,2.5,3.0
Paula Turnip,4.0,1.0,3.5,3.0
Hahn Fox,4.0,2.5,4.0,2.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
August Sky,0.896214,1.023915,0.872428,0.931996,0.703057,0.321205,0.386864,0.001000,0.389760
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Denzel Scott,0.850411,0.963515,0.849616,0.566107,0.627399,0.119743,0.673793,0.007747,0.949707
Peanutiel Duffy,0.841420,1.038584,0.738454,0.996131,0.670202,0.418180,0.343720,0.100000,0.240592
Rat Mason,0.831824,1.089116,0.635958,0.783859,0.732174,0.207444,0.899090,0.019604,0.485050
Paula Turnip,0.816815,0.766132,1.017299,0.583585,0.885149,0.544858,0.284335,0.100000,0.261251
Hahn Fox,0.806886,0.889348,0.971066,0.656774,0.409341,0.653772,0.607895,0.160000,0.386523


In [98]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:10]))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:10]), maxVal=1))

####Henry Marshallow

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,4.0,2.5,4.5
Elvis Figueroa,3.5,5.5,2.0,4.5
Xandra Pancakes,2.5,5.0,4.0,4.5
Sixpack Santiago,2.0,5.0,4.5,3.0
Rafael Davids,3.5,5.0,2.0,3.5
----eide- Bend-e,3.5,4.5,4.5,2.5
Cory Twelve,4.0,4.5,3.5,2.5
Yummy Elliott,3.5,4.5,4.0,4.0
Engine Eberhardt,3.5,4.5,5.0,4.0
Yurts Buttercup,4.0,4.5,3.5,4.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Elvis Figueroa,1.096633,1.159675,1.136143,1.072682,0.786089,0.461405,0.721203
Xandra Pancakes,0.983680,1.082650,1.004702,0.855130,0.854402,0.470938,0.725690
Sixpack Santiago,0.969910,0.765921,1.199201,1.531857,0.747720,0.826116,0.559834
Rafael Davids,0.950434,0.821531,1.097610,1.057661,1.006934,1.045070,1.161455
----eide- Bend-e,0.944823,0.944301,1.093810,0.979122,0.575652,0.799962,0.643485
Cory Twelve,0.942071,1.059549,1.062618,0.701061,0.544233,1.050049,0.258487
Yummy Elliott,0.912388,0.798342,1.147276,1.108293,0.617898,0.948697,0.932231
Engine Eberhardt,0.870702,1.153561,0.757394,0.612956,0.819308,0.806215,0.779997
Yurts Buttercup,0.852575,0.987203,1.095826,0.627572,0.287007,1.223968,0.584248


####Nerd Pacheco

,Batting,Pitching,Baserunning,Defense
Nerd Pacheco,4.5,4.0,4.5,3.0
Burke Gonzales,3.5,6.0,4.0,4.0
Elvis Figueroa,3.5,5.5,2.0,4.5
Xandra Pancakes,2.5,5.0,4.0,4.5
Sixpack Santiago,2.0,5.0,4.5,3.0
Rigby Friedrich,3.5,5.0,4.5,4.0
Rafael Davids,3.5,5.0,2.0,3.5
----eide- Bend-e,3.5,4.5,4.5,2.5
Cory Twelve,4.0,4.5,3.5,2.5
Yummy Elliott,3.5,4.5,4.0,4.0


...

,Batting,Pitching,Baserunning,Defense
Ruslan Greatness,5.0,4.0,5.0,5.0
Pitching Machine,4.5,6.5,3.5,4.0
Burke Gonzales,3.5,6.0,4.0,4.0
Lou Roseheart,4.0,6.0,5.5,4.5
Qais Dogwalker,4.5,6.0,5.5,5.0
Rigby Friedrich,3.5,5.0,4.5,4.0
Knight Tri---h--t,5.5,4.5,4.0,5.0
Fitzgerald Blackburn,5.5,4.5,2.5,4.5
Wesley Poole,3.5,4.0,5.5,5.5
Sexton Wheerer,4.5,4.0,3.5,5.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Ruslan Greatness,0.819339,0.770870,0.786481,1.269963,1.026832,0.629742,1.420762
Pitching Machine,1.253365,1.412498,1.269259,1.241936,0.523310,0.671280,0.901447
Burke Gonzales,1.176989,1.171639,1.341412,0.879956,0.870391,0.973638,0.789646
Lou Roseheart,1.173663,1.199089,1.014134,1.171751,1.394565,1.307418,1.255015
Qais Dogwalker,1.153869,1.071425,1.137700,1.358705,1.152111,0.882765,0.610121
Rigby Friedrich,0.961982,1.119728,1.004998,0.497562,1.143943,0.785381,0.584288
Knight Tri---h--t,0.919301,1.212409,0.615961,1.063965,1.186994,0.593097,0.480777
Fitzgerald Blackburn,0.894261,0.859707,1.024373,1.000059,0.774466,0.444692,0.794618
Wesley Poole,0.828930,1.036001,0.861256,0.398420,1.032953,0.650169,0.834712
Sexton Wheerer,0.826895,0.798889,0.906990,0.837579,0.933111,0.810487,0.820770


####Zack Sanders

,Batting,Pitching,Baserunning,Defense
Zack Sanders,3.0,2.5,4.0,4.5
Elvis Figueroa,3.5,5.5,2.0,4.5
Sixpack Santiago,2.0,5.0,4.5,3.0
Rafael Davids,3.5,5.0,2.0,3.5
----eide- Bend-e,3.5,4.5,4.5,2.5
Cory Twelve,4.0,4.5,3.5,2.5
Yurts Buttercup,4.0,4.5,3.5,4.0
Wyatt Pothos,2.5,4.0,3.5,3.0
Cell Barajas,3.0,4.0,2.5,3.0
Rivers Rosa,3.5,4.0,4.5,4.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Zack Sanders,0.529257,0.371443,0.968237,0.733656,0.519537,0.522018,0.671376
Elvis Figueroa,1.096633,1.159675,1.136143,1.072682,0.786089,0.461405,0.721203
Sixpack Santiago,0.969910,0.765921,1.199201,1.531857,0.747720,0.826116,0.559834
Rafael Davids,0.950434,0.821531,1.097610,1.057661,1.006934,1.045070,1.161455
----eide- Bend-e,0.944823,0.944301,1.093810,0.979122,0.575652,0.799962,0.643485
Cory Twelve,0.942071,1.059549,1.062618,0.701061,0.544233,1.050049,0.258487
Yurts Buttercup,0.852575,0.987203,1.095826,0.627572,0.287007,1.223968,0.584248
Wyatt Pothos,0.844384,0.701425,1.063023,1.132335,0.691922,1.079506,0.799864
Cell Barajas,0.819420,0.927790,0.968070,0.718913,0.614807,0.132356,0.139887
Rivers Rosa,0.808070,0.612831,1.154332,1.206265,0.735692,0.396654,0.657722


### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [99]:
shadows = pies.bullpen + pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

display(get_stars([worst_batter] + best_batting_shadows))
display(set_heatmap(get_batting_stlats([worst_batter] + best_batting_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Yusef Puddles,2.0,2.5,3.0,3.5
Haruta Byrd,3.5,2.5,2.5,1.5
Javier Lotus,3.5,1.0,3.5,4.0
August Obrien,3.5,1.0,3.5,2.0
Doc Anice,3.0,3.5,3.0,4.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958
Doc Anice,0.620593,0.759195,0.392327,0.792557,0.733223,0.174657,1.058193,0.329574,0.817492


**Pitching:**

In [100]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

display(get_stars([worst_pitcher] + best_pitching_shadows))
display(set_heatmap(get_pitching_stlats([worst_pitcher] + best_pitching_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Zack Sanders,3.0,2.5,4.0,4.5
Lucy Tokkan,2.0,3.5,1.0,3.0
Doc Anice,3.0,3.5,3.0,4.5
Twofurious Puddles,2.5,3.5,2.0,3.5
Case Lancaster,0.0,3.5,2.0,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Zack Sanders,0.529257,0.371443,0.968237,0.733656,0.519537,0.522018,0.671376
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Doc Anice,0.734279,1.076739,0.487964,0.959145,0.632001,0.765347,0.305586
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894
Case Lancaster,0.658365,0.867044,0.829197,0.190119,0.825333,0.868251,0.082117


### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving our best hitting pitcher or best pitching hitter into their preferred
position.

**Batting**

In [101]:
bhp = best_hitting_pitcher(pies)
bph = best_pitching_hitter(pies)

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "bench")))

Moving Ruslan Greatness to the Lineup

,Pies
batting_change,+2.0571%
pitching_change,-2.6241%
baserunning_change,+4.4490%
defense_change,+4.2182%


Moving Yusef Puddles to the Shadows

,Pies
batting_change,+5.9494%
pitching_change,+0.0000%
baserunning_change,+1.8896%
defense_change,-0.7445%


**Pitching**

In [102]:
display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "bullpen")))

Moving Jaxon Buck-ey to the Rotation

,Pies
batting_change,-8.8672%
pitching_change,+0.3368%
baserunning_change,-3.4884%
defense_change,+0.2810%


Moving Zack Sanders to the Shadows

,Pies
batting_change,+0.0000%
pitching_change,+4.6280%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Transfuse
**Boost a Player on your Team by 15% to 30% in a Stat Category of your choice, but Impair them in a Random other Category.**

The stat that is negatively impacted is random. It is generally better to boost pitchers, as only the pitching stats are
used for that position. See below for how this would affect various team members at the 2 extreme percentages. This will
only affect one player.

**15% Change:**

In [103]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.15)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.15)), maxVal=1.0))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,4.885272,0.719972
Beasley Day,4.274532,4.998553,0.724021
Huber Frumple,4.287244,5.039095,0.751851
Jaxon Buck-ey,6.592812,7.226397,0.633585
Lang Richardson,3.162214,3.907698,0.745485
Yusef Puddles,2.147811,3.068746,0.920935
Usurper Violet,2.898577,3.704516,0.805939


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.977054,1.169391,0.996540,0.800666,0.725519,0.445517,0.724554,0.010000,0.855869
Beasley Day,0.999711,1.195160,1.054890,0.522953,0.697377,0.061371,0.883784,0.010000,1.159991
Huber Frumple,1.007819,0.657458,1.460733,1.082690,1.371501,0.076988,0.835091,0.010000,1.263159
Jaxon Buck-ey,1.445279,1.717382,1.192101,1.896709,2.080045,0.010000,2.136793,0.010000,1.543504
Lang Richardson,0.781540,0.962722,0.550621,0.772843,1.091821,0.055246,0.648400,0.010000,0.701015
Yusef Puddles,0.613749,1.183698,0.334577,0.825196,0.301908,0.480778,0.260821,0.010000,1.160015
Usurper Violet,0.740903,1.044648,0.443503,1.172726,1.019795,0.419699,0.433561,0.010000,0.637176


In [104]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.15)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.15)), maxVal=1))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,4.774535,0.932985
Nerd Pacheco,4.180600,5.151709,0.971109
Tiana Takahashi,3.094250,4.393265,1.299014
Ruslan Greatness,4.096695,4.976003,0.879308
Zack Sanders,2.646285,3.525263,0.878978


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.954907,0.641270,1.321077,1.578524,0.972605,0.958351,1.026106
Nerd Pacheco,1.030342,1.237720,1.216351,0.422113,0.758134,1.082212,0.634500
Tiana Takahashi,0.878653,1.060065,1.111592,0.866677,0.178954,0.746122,1.095940
Ruslan Greatness,0.995201,0.920870,0.936481,1.419963,1.176832,0.779742,1.570762
Zack Sanders,0.705053,0.521443,1.118237,0.883656,0.669537,0.672018,0.821376


**30% Change:**

In [105]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.30)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.30)), maxVal=1))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,5.585918,1.420618
Beasley Day,4.274532,5.676802,1.402269
Huber Frumple,4.287244,5.741328,1.454084
Jaxon Buck-ey,6.592812,7.852389,1.259577
Lang Richardson,3.162214,4.606233,1.444019
Yusef Puddles,2.147811,3.885398,1.737587
Usurper Violet,2.898577,4.460398,1.561822


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.117184,1.319391,1.146540,0.950666,0.875519,0.295517,0.874554,0.010000,1.005869
Beasley Day,1.135360,1.345160,1.204890,0.672953,0.847377,0.010000,1.033784,0.010000,1.309991
Huber Frumple,1.148266,0.807458,1.610733,1.232690,1.521501,0.010000,0.985091,0.010000,1.413159
Jaxon Buck-ey,1.570478,1.867382,1.342101,2.046709,2.230045,0.010000,2.286793,0.010000,1.693504
Lang Richardson,0.921247,1.112722,0.700621,0.922843,1.241821,0.010000,0.798400,0.010000,0.851015
Yusef Puddles,0.777080,1.333698,0.484577,0.975196,0.451908,0.330778,0.410821,0.010000,1.310015
Usurper Violet,0.892080,1.194648,0.593503,1.322726,1.169795,0.269699,0.583561,0.010000,0.787176


In [106]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.30)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.30)), maxVal=1))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,5.714489,1.872939
Nerd Pacheco,4.180600,6.110125,1.929525
Tiana Takahashi,3.094250,5.399357,2.305106
Ruslan Greatness,4.096695,5.876536,1.779841
Zack Sanders,2.646285,4.417130,1.770845


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,1.142898,0.791270,1.471077,1.728524,1.122605,1.108351,1.176106
Nerd Pacheco,1.222025,1.387720,1.366351,0.572113,0.908134,1.232212,0.784500
Tiana Takahashi,1.079871,1.210065,1.261592,1.016677,0.328954,0.896122,1.245940
Ruslan Greatness,1.175307,1.070870,1.086481,1.569963,1.326832,0.929742,1.720762
Zack Sanders,0.883426,0.671443,1.268237,1.033656,0.819537,0.822018,0.971376


### Alternate Trust
**Call in an Alternate for a Player on your Team, with a mystery Benefit.**

An Alternate is a player reroll and the Benefit is an unknown modification. Below shows the change in Team Average Stars from rerolling our worst
player, assuming they would be replaced with the Statistically Average player.

In [107]:
worst_pie = sort_overall(pies, 1)[0]

display(Markdown(f"Rerolling {worst_pie.name}"))
if worst_pie in pies.rotation:
    out = replace_player(pies, worst_pie, pitch_star=NEW_PLAYER["pitching"])
else:
    out = replace_player(pies, worst_pie, bat_star=NEW_PLAYER["batting"], run_star=NEW_PLAYER["baserunning"], def_star=NEW_PLAYER["defense"])
pandas.DataFrame(out)

Rerolling Yusef Puddles

,Pies
batting_change,-0.4223%
pitching_change,+0.0000%
baserunning_change,-0.9070%
defense_change,-3.1326%


### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [108]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

,mods
Beasley Day,[Minimalist]
Jaxon Buck-ey,[Fire Eater]
Henry Marshallow,[Maximalist]
Ruslan Greatness,[Friend of Crows]


### Enrich
**Pick a modification for your Team next season.**

This will gives the option of choosing between
* The Fourth Strike: Those with the Fourth Strike will get an extra strike in each at bat.
* High Pressure: This Team will play 25% better in Flooding weather when runners are on base.
* Life of the Party: This team gets 10% more from their Party Time stat boosts.
* Home Field Advantage: Teams with Home Field Advantage will start each home game with one run.
* Affinity for Crows: Players with Affinity for Crows will hit and pitch 50% better during Birds weather.
* Traveling: Traveling teams will play 5% better in Away games.

Seasonal Modifications have been mixed in the past. For Fourth Strike Analysis, see the appropriate
[SIBR Paper](https://research.blaseball-reference.com/Fourth_Strike.pdf).

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### Early Riser
**A random Player on your Team will be a permanent Earlbird.**

Earlbird players will Overperform (20% boost) in the first 27 games of the season.

### Middle of the Pack
**A random Player on your Team will be permanently Middling.**

Middling players will Overperform (20% boost) in the middle 45 games of the season.

### Fashionably Late
**A random Player on your Team will be permanently Late to the Party.**

Late to the Party players will Overperform (20% boost) in the last 27 games of the season.

### Posterizer
**A random Player on your Team will be permanently Ambitious.**

The effect of Ambitious is unknown.

### Supply Run
**All Players on your Team with an available Item slot (empty or broken) will open a Generic Bat Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Cap Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Shoe Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Glove Crate.**

This would provide additional Generic Bats, Caps, Shoes, or Gloves to the following players:

TODO: Get players with free item slots

### Handcrafted Drop
**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Bat Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Cap Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Shoe Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Glove Crate.**

This would provide additional Special Bats, Caps, Shoes, or Gloves to one of the following players:

TODO: Get players with free item slots

### Skateboard
**A Random Hitter on your Team gets a Skateboard.**

The effect of a Skateboard (other than being radical 🛹) is unknown.

### Fired Up
**Get Fired Up. Give your Team Fire Blood.**

This would make all our players have Fire Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### Mind Trick
**Give your Team Psychic Blood.**

This would make all our players have Psychic Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### Power Chaaarge
**Chaaarge! Give your Team AAA Blood.**

This would make all our players have AAA Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### Clean Up
**Move the Player with the highest Divinity on your Team to the Cleanup spot (the 4th spot in your Lineup).**

See below for the new lineup order.

In [109]:
new_lineup = pies.lineup.copy()
new_lineup.sort(key=lambda x: x.divinity, reverse=True)
best_divinity = new_lineup[0]

new_lineup = pies.lineup.copy()
new_lineup.insert(3, new_lineup.pop(new_lineup.index(best_divinity)))
get_stars(new_lineup)

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Jaxon Buck-ey,6.5,3.5,4.5,3.5
Huber Frumple,4.5,1.5,4.0,3.0
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,3.5


### Weakest Link
**Move the Worst Defender in your Lineup to the Shadows.**

See below for the removed player and the change in team stats.

In [110]:
new_lineup.sort(key=lambda x: x.defense_rating)
worst_defender = new_lineup[0]

display(Markdown(f"Moving {worst_defender.name} to the Shadows"))
pandas.DataFrame(remove_player(pies, worst_defender))

Moving Huber Frumple to the Shadows

,Pies
batting_change,-1.1820%
pitching_change,+0.0000%
baserunning_change,-2.1862%
defense_change,+1.5639%


### Time to Shine
**Promote the Best Hitter in your Team's Shadows to a random position in the Lineup.**

See below for new player and the change in team stats.

In [111]:
new_lineup = pies.bench + pies.bullpen
new_lineup.sort(key=lambda x: x.batting_rating, reverse=True)
best_shadows_batter = new_lineup[0]

display(Markdown(f"Moving {best_shadows_batter.name} to the Lineup"))
display(get_stars([best_shadows_batter] + pies.lineup))
display(set_heatmap(get_batting_stlats([best_shadows_batter] + pies.lineup), maxVal=1))
pandas.DataFrame(move_player(pies, best_shadows_batter, "lineup"))

Moving Haruta Byrd to the Lineup

,Batting,Pitching,Baserunning,Defense
Haruta Byrd,3.5,2.5,2.5,1.5
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,1.5,4.0,3.0
Jaxon Buck-ey,6.5,3.5,4.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,3.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Eduardo Woodman,0.833060,1.019391,0.846540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.854906,1.045160,0.904890,0.372953,0.547377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.857449,0.507458,1.310733,0.932690,1.221501,0.226988,0.685091,0.041371,1.113159
Jaxon Buck-ey,1.318562,1.567382,1.042101,1.746709,1.930045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.632443,0.812722,0.400621,0.622843,0.941821,0.205246,0.498400,0.001000,0.551015
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176


,Pies
batting_change,-0.7767%
pitching_change,+0.0000%
baserunning_change,-2.3474%
defense_change,-4.2528%


### Sweet Relief
**Promote the Best Pitcher in your Team's Shadows to a random position in the Rotation.**

See below for new player and the change in team stats.

In [112]:
new_lineup.sort(key=lambda x: x.pitching_rating, reverse=True)
best_shadows_pitcher = new_lineup[0]

display(Markdown(f"Moving {best_shadows_pitcher.name} to the Rotation"))
display(get_stars([best_shadows_pitcher] + pies.rotation))
display(set_heatmap(get_pitching_stlats([best_shadows_pitcher] + pies.rotation), maxVal=1))
pandas.DataFrame(move_player(pies, best_shadows_pitcher, "rotation"))

Moving Lucy Tokkan to the Rotation

,Batting,Pitching,Baserunning,Defense
Lucy Tokkan,2.0,3.5,1.0,3.0
Henry Marshallow,3.5,4.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Tiana Takahashi,1.0,3.0,2.5,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Zack Sanders,3.0,2.5,4.0,4.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Nerd Pacheco,0.836120,1.087720,1.066351,0.272113,0.608134,0.932212,0.484500
Tiana Takahashi,0.618850,0.910065,0.961592,0.716677,0.028954,0.596122,0.945940
Ruslan Greatness,0.819339,0.770870,0.786481,1.269963,1.026832,0.629742,1.420762
Zack Sanders,0.529257,0.371443,0.968237,0.733656,0.519537,0.522018,0.671376


,Pies
batting_change,+0.0000%
pitching_change,+0.4562%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Shadow Practice
**Your Team's Shadows will have their Defense/Baserunning/Pitching/Hitting Boosted from 1% to 12%.**

These blessings increase the team-wide stats or our shadow players. See below for worst and best case rolls. This
assumes that it will only boost the respective shadow positions, so batting will only affect the bench, etc.

#### Shadow Batting Practice

**Worst Case**

In [113]:
table, total, avg = improve_team_batting_table(pies, 0.01, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 0.465582514784211

Change in Average stars: 0.0775970857973685

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,2.414010,0.060718
August Obrien,3.408347,3.470699,0.062352
Mcfarland Vargas,1.383943,1.476773,0.092829
Twofurious Puddles,2.635533,2.722015,0.086482
Mummy Cantburn,1.390958,1.477752,0.086794
Doginic Burgertoes,2.079285,2.155693,0.076407


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.482802,0.312171,0.718122,0.961649,0.112259,0.265678,0.342170,0.090000,0.553764
August Obrien,0.694140,0.651217,0.937569,0.923891,0.502708,0.913854,0.566849,0.090000,0.473958
Mcfarland Vargas,0.295355,0.127084,0.786150,0.317714,0.185366,0.974279,0.427832,0.090000,0.561343
Twofurious Puddles,0.544403,0.587625,0.490623,0.867061,0.986273,0.802102,0.019182,0.090000,0.484513
Mummy Cantburn,0.295550,0.105592,0.586609,0.957265,0.535890,0.906564,0.022553,0.090000,0.937160
Doginic Burgertoes,0.431139,0.186724,0.994426,0.115629,0.944521,0.202919,0.048098,0.751237,0.314905


**Best Case**

In [114]:
table, total, avg = improve_team_batting_table(pies, 0.12, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.12, shadows=True)), maxVal=1))


Change in Total stars: 4.548014966625795

Change in Average stars: 0.7580024944376325

,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,3.033053,0.679762
August Obrien,3.408347,4.082129,0.673782
Mcfarland Vargas,1.383943,2.225020,0.841077
Twofurious Puddles,2.635533,3.354184,0.718652
Mummy Cantburn,1.390958,2.220729,0.829771
Doginic Burgertoes,2.079285,2.884257,0.804971


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.606611,0.422171,0.828122,1.071649,0.222259,0.155678,0.452170,0.010000,0.663764
August Obrien,0.816426,0.761217,1.047569,1.033891,0.612708,0.803854,0.676849,0.010000,0.583958
Mcfarland Vargas,0.445004,0.237084,0.896150,0.427714,0.295366,0.864279,0.537832,0.010000,0.671343
Twofurious Puddles,0.670837,0.697625,0.600623,0.977061,1.096273,0.692102,0.129182,0.010000,0.594513
Mummy Cantburn,0.444146,0.215592,0.696609,1.067265,0.645890,0.796564,0.132553,0.010000,1.047160
Doginic Burgertoes,0.576851,0.296724,1.104426,0.225629,1.054521,0.092919,0.158098,0.641237,0.424905


#### Shadow Pitching Practice

**Worst Case**

In [115]:
table, total, avg = improve_team_pitching_table(pies, 0.01, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 0.6238904095508968

Change in Average stars: 0.06932115661676631

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Castillo Logan,3.022344,3.093824,0.071480
Bambi Perez,0.340030,0.431085,0.091055
Jesús Rodriguez,1.281753,1.337202,0.055449
Jefferson Delacruz,0.583451,0.684939,0.101488
Case Lancaster,3.291826,3.357347,0.065521
Haruta Byrd,2.728771,2.789924,0.061153
Javier Lotus,0.765838,0.818349,0.052510
Doc Anice,3.671394,3.731306,0.059912
Lucy Tokkan,3.708740,3.774062,0.065322


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Castillo Logan,0.618765,0.972056,0.689044,0.176984,1.004242,0.100067,0.240832
Bambi Perez,0.086217,0.028422,0.488632,0.284202,0.161506,0.592147,0.120881
Jesús Rodriguez,0.267440,0.201237,0.689646,0.294989,0.196662,0.401038,0.818921
Jefferson Delacruz,0.136988,0.215741,0.195705,0.396775,0.014881,0.693663,0.107056
Case Lancaster,0.671469,0.877044,0.839197,0.200119,0.835333,0.878251,0.092117
Haruta Byrd,0.557985,0.756009,0.694712,0.199109,0.612043,0.765252,0.370515
Javier Lotus,0.163670,0.117195,0.280303,0.280723,0.808413,0.499265,0.631886
Doc Anice,0.746261,1.086739,0.497964,0.969145,0.642001,0.775347,0.315586
Lucy Tokkan,0.754812,0.911966,1.005562,0.273758,0.983020,0.190980,0.331805


**Best Case**

In [116]:
table, total, avg = improve_team_pitching_table(pies, 0.12, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.12, shadows=True)), maxVal=1))

Change in Total stars: 6.598317364955489

Change in Average stars: 0.7331463738839432

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Castillo Logan,3.022344,3.833175,0.810831
Bambi Perez,0.340030,1.146391,0.806361
Jesús Rodriguez,1.281753,1.946115,0.664362
Jefferson Delacruz,0.583451,1.295765,0.712314
Case Lancaster,3.291826,4.057331,0.765505
Haruta Byrd,2.728771,3.450752,0.721981
Javier Lotus,0.765838,1.391343,0.625504
Doc Anice,3.671394,4.393214,0.721820
Lucy Tokkan,3.708740,4.478378,0.769638


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Castillo Logan,0.766635,1.082056,0.799044,0.286984,1.114242,0.210067,0.350832
Bambi Perez,0.229278,0.138422,0.598632,0.394202,0.271506,0.702147,0.230881
Jesús Rodriguez,0.389223,0.311237,0.799646,0.404989,0.306662,0.511038,0.928921
Jefferson Delacruz,0.259153,0.325741,0.305705,0.506775,0.124881,0.803663,0.217056
Case Lancaster,0.811466,0.987044,0.949197,0.310119,0.945333,0.988251,0.202117
Haruta Byrd,0.690150,0.866009,0.804712,0.309109,0.722043,0.875252,0.480515
Javier Lotus,0.278269,0.227195,0.390303,0.390723,0.918413,0.609265,0.741886
Doc Anice,0.878643,1.196739,0.607964,1.079145,0.752001,0.885347,0.425586
Lucy Tokkan,0.895676,1.021966,1.115562,0.383758,1.093020,0.300980,0.441805


#### Shadow Running Practice

**Worst Case**

In [117]:
table, total, avg = improve_team_baserunning_table(pies, 0.01, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 0.3779591112420766

Change in Average stars: 0.06299318520701276

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,3.171746,0.055049
August Obrien,3.318856,3.378726,0.059870
Mcfarland Vargas,3.783245,3.831944,0.048700
Twofurious Puddles,1.817993,1.877135,0.059142
Mummy Cantburn,2.021513,2.083616,0.062102
Doginic Burgertoes,1.779564,1.872660,0.093096


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,0.634349,0.850831,0.594640,0.495051,0.186399,0.431241
August Obrien,0.675745,0.976470,0.150389,0.711812,0.491149,0.425344
Mcfarland Vargas,0.766389,0.932765,0.567085,0.822168,0.492377,0.431246
Twofurious Puddles,0.375427,0.203390,0.388033,0.524860,0.898752,0.873079
Mummy Cantburn,0.416723,0.298250,0.842320,0.949393,0.108762,0.769435
Doginic Burgertoes,0.374532,0.332288,0.037095,0.627854,0.637448,0.903013


**Best Case**

In [118]:
table, total, avg = improve_team_baserunning_table(pies, 0.12, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(pies, 0.12, shadows=True)), maxVal=1))

Change in Total stars: 4.0516944755187625

Change in Average stars: 0.6752824125864604

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,3.747971,0.631273
August Obrien,3.318856,3.988228,0.669372
Mcfarland Vargas,3.783245,4.359164,0.575919
Twofurious Puddles,1.817993,2.492548,0.674555
Mummy Cantburn,2.021513,2.711657,0.690144
Doginic Burgertoes,1.779564,2.589995,0.810431


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,0.749594,0.960831,0.704640,0.605051,0.296399,0.541241
August Obrien,0.797646,1.086470,0.260389,0.821812,0.601149,0.535344
Mcfarland Vargas,0.871833,1.042765,0.677085,0.932168,0.602377,0.541246
Twofurious Puddles,0.498510,0.313390,0.498033,0.634860,1.008752,0.983079
Mummy Cantburn,0.542331,0.408250,0.952320,1.059393,0.218762,0.879435
Doginic Burgertoes,0.517999,0.442288,0.147095,0.737854,0.747448,1.013013


#### Shadow Defense Practice

**Worst Case**

In [119]:
table, total, avg = improve_team_defense_table(pies, 0.01, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 1.125697690461016

Change in Average stars: 0.07504651269740108

,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,2.718599,0.054187
August Obrien,2.035102,2.125446,0.090344
Mcfarland Vargas,2.071032,2.162907,0.091874
Twofurious Puddles,3.610790,3.655879,0.045089
Mummy Cantburn,3.289650,3.332702,0.043052
Doginic Burgertoes,1.538638,1.636112,0.097474
Castillo Logan,2.996505,3.049184,0.052679
Bambi Perez,2.859667,2.923939,0.064272
Jesús Rodriguez,1.417111,1.614089,0.196978
Jefferson Delacruz,3.914262,3.955411,0.041149


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.543720,0.198840,0.353662,0.915908,0.526713,0.946550
August Obrien,0.425089,0.064615,0.493368,0.847103,0.337928,0.662261
Mcfarland Vargas,0.432581,0.064401,0.366182,0.600229,0.751915,0.914134
Twofurious Puddles,0.731176,0.857265,0.919861,0.270410,0.665028,0.390553
Mummy Cantburn,0.666540,0.601982,0.406224,0.526128,0.635545,0.865624
Doginic Burgertoes,0.327222,0.057259,0.874391,0.425999,0.128550,0.102532
Castillo Logan,0.609837,0.759614,0.201755,0.727648,0.501621,0.829861
Bambi Perez,0.584788,0.732095,0.807496,0.744400,0.151465,0.118700
Jesús Rodriguez,0.322818,0.277421,0.105104,0.971620,0.944389,0.015755
Jefferson Delacruz,0.791082,0.986397,0.431596,0.814808,0.794419,0.818188


**Best Case**

In [120]:
table, total, avg = improve_team_defense_table(pies, 0.12, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(pies, 0.12, shadows=True)), maxVal=1))

Change in Total stars: 10.068062713515326

Change in Average stars: 0.6712041809010217

,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,3.266263,0.601851
August Obrien,2.035102,2.854276,0.819174
Mcfarland Vargas,2.071032,2.901058,0.830026
Twofurious Puddles,3.610790,4.126072,0.515282
Mummy Cantburn,3.289650,3.789322,0.499672
Doginic Burgertoes,1.538638,2.430513,0.891874
Castillo Logan,2.996505,3.579799,0.583295
Bambi Perez,2.859667,3.526931,0.667264
Jesús Rodriguez,1.417111,2.504935,1.087824
Jefferson Delacruz,3.914262,4.392425,0.478163


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.653253,0.308840,0.463662,1.025908,0.636713,1.056550
August Obrien,0.570855,0.174615,0.603368,0.957103,0.447928,0.772261
Mcfarland Vargas,0.580212,0.174401,0.476182,0.710229,0.861915,1.024134
Twofurious Puddles,0.825214,0.967265,1.029861,0.380410,0.775028,0.500553
Mummy Cantburn,0.757864,0.711982,0.516224,0.636128,0.745545,0.975624
Doginic Burgertoes,0.486103,0.167259,0.984391,0.535999,0.238550,0.212532
Castillo Logan,0.715960,0.869614,0.311755,0.837648,0.611621,0.939861
Bambi Perez,0.705386,0.842095,0.917496,0.854400,0.261465,0.228700
Jesús Rodriguez,0.500987,0.387421,0.215104,1.081620,1.054389,0.125755
Jefferson Delacruz,0.878485,1.096397,0.541596,0.924808,0.904419,0.928188


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
